In [34]:
from scipy.io import loadmat


In [35]:
cars_train_annos = loadmat('devkit/cars_train_annos.mat')

In [36]:
cars_test_annos = loadmat('devkit/cars_test_annos.mat')

In [37]:
def to_list_train(anno):
    return [anno[0][0][0], anno[1][0][0], anno[2][0][0], anno[3][0][0], anno[4][0][0]], anno[5][0]

In [38]:
def to_list_test(anno):
    return [anno[0][0][0], anno[1][0][0], anno[2][0][0], anno[3][0][0], anno[4][0][0]]

In [39]:
import cv2

In [40]:
import matplotlib.pyplot as plt

here 0 1 - top of bb, 2,3 - bot of bb, 4 - label 

In [52]:
to_list(test)

([36, 116, 868, 587, 3], '00002.jpg')

In [52]:
def create_keras_folders(annotation, path, out_path):
    for anno in annotation:
        box_and_label = annotation[anno]

        b0=box_and_label[0]
        b1=box_and_label[1]
        b2=box_and_label[2]
        b3=box_and_label[3]

        label = box_and_label[4]
        if not os.path.exists(out_path + "/" +  str(label)):
            os.mkdir(out_path + "/" + str(label))

        img = cv2.imread(path + "/" + anno)
        if img is None:
            continue
        res_path = out_path + "/" + str(label) + "/" + anno
        cv2.imwrite(res_path, img[b1:b3, b0:b2] )

In [3]:
import os

In [23]:
names = os.listdir('cars_train')

In [6]:
from sklearn.model_selection import train_test_split

In [15]:
train, val = train_test_split(names)

In [8]:
from shutil import move

In [17]:
for t in train:
    if os.path.isfile('cars_train/' + t):
        move('cars_train/' + t, 'cars_train/train/' + t)

In [30]:
def prepare_annotation_train(raw_annotation):
    annotations_len = len(raw_annotation['annotations'][0])
    annos = {}
    for i in range(0,annotations_len):
        points, file = to_list_train(raw_annotation['annotations'][0][i])
        annos[file] = points
        
    return annos

In [31]:
def prepare_annotation_test(raw_annotation):
    annotations_len = len(raw_annotation['annotations'][0])
    annos = {}
    for i in range(0,annotations_len):
        points = to_list_test(raw_annotation['annotations'][0][i])
        
    return annos


In [41]:
train_anno = prepare_annotation_train(cars_train_annos)

In [42]:
test_anno = prepare_annotation_test(cars_test_annos)

In [53]:
create_keras_folders(train_anno, "cars_train/train/", "data/train/")

In [54]:
create_keras_folders(train_anno, "cars_train/val/", "data/val/")

In [1]:
import tensorflow

In [1]:
from keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
def get_generator():
    img_size = 128
    train_gen = ImageDataGenerator(samplewise_center=True,
                                   samplewise_std_normalization=True,
                                  rotation_range=20,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=True)
    
    val_gen = ImageDataGenerator(samplewise_center=True,
                                   samplewise_std_normalization=True)
    
    train  = train_gen.flow_from_directory('data/train',
                                           target_size=(img_size, img_size),
                                          batch_size=1)
    val = val_gen.flow_from_directory('data/val',
                                      target_size=(img_size, img_size),
                                     batch_size=1)
    return train, val


In [3]:
train, val  = get_generator()

Found 7101 images belonging to 196 classes.
Found 1043 images belonging to 196 classes.


In [4]:
base = VGG16(weights='imagenet', include_top=False)

In [5]:
x = base.output
x = Dropout(0.1)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(384, activation='relu')(x)
predictions = Dense(196, activation='softmax')(x)

In [6]:
model = Model(inputs=base.input, outputs=predictions)

In [7]:
batch_size = 2

In [8]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
early_stop = EarlyStopping(monitor='val_acc', patience=20)

In [10]:
model.fit_generator(
    generator=train, 
    epochs=200, 
    steps_per_epoch=int(train.samples / batch_size),
    validation_data=val,
    validation_steps=int(val.samples / batch_size))

Epoch 1/200
3550/3550 [==============================] - 445s - loss: 5.2850 - acc: 0.0048 - val_loss: 5.2797 - val_acc: 0.0038
Epoch 2/200
3550/3550 [==============================] - 443s - loss: 5.2796 - acc: 0.0068 - val_loss: 5.2788 - val_acc: 0.0096
Epoch 3/200
3550/3550 [==============================] - 444s - loss: 5.2774 - acc: 0.0073 - val_loss: 5.2819 - val_acc: 0.0058
Epoch 4/200
3550/3550 [==============================] - 444s - loss: 5.2776 - acc: 0.0093 - val_loss: 5.2793 - val_acc: 0.0115
Epoch 5/200
3550/3550 [==============================] - 444s - loss: 5.2766 - acc: 0.0090 - val_loss: 5.2805 - val_acc: 0.0077
Epoch 6/200
3550/3550 [==============================] - 443s - loss: 5.2767 - acc: 0.0076 - val_loss: 5.2842 - val_acc: 0.0096
Epoch 7/200
3550/3550 [==============================] - 435s - loss: 5.2762 - acc: 0.0076 - val_loss: 5.2843 - val_acc: 0.0038
Epoch 8/200
3550/3550 [==============================] - 435s - loss: 5.2753 - acc: 0.0090 - val_loss: 5

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,512,512]
	 [[Node: training/Adam/mul_93 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](Adam/beta_2/read, training/Adam/Variable_48/read)]]

Caused by op 'training/Adam/mul_93', defined at:
  File "/home/lyan/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/lyan/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/lyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/lyan/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/lyan/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/lyan/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
